<a href="https://colab.research.google.com/github/pepe-olivert/unicc_project/blob/HugoA/EvidenceSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=83c31b35219b202fd40817be40fc1af0ed26fac2d3506612a8f50a4a970c9c26
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

[[ 6.76569417e-02  6.34960085e-02  4.87131625e-02  7.93049410e-02
   3.74480858e-02  2.65283487e-03  3.93749997e-02 -7.09846290e-03
   5.93614094e-02  3.15369666e-02  6.00980744e-02 -5.29052280e-02
   4.06067707e-02 -2.59308703e-02  2.98428256e-02  1.12688239e-03
   7.35148713e-02 -5.03819063e-02 -1.22386619e-01  2.37028785e-02
   2.97265593e-02  4.24768329e-02  2.56337505e-02  1.99517841e-03
  -5.69190383e-02 -2.71597970e-02 -3.29035111e-02  6.60248771e-02
   1.19007193e-01 -4.58790921e-02 -7.26214126e-02 -3.25840227e-02
   5.23413047e-02  4.50553037e-02  8.25307053e-03  3.67024168e-02
  -1.39415748e-02  6.53918162e-02 -2.64272206e-02  2.06408804e-04
  -1.36643089e-02 -3.62810753e-02 -1.95043404e-02 -2.89737899e-02
   3.94269899e-02 -8.84090587e-02  2.62428867e-03  1.36713544e-02
   4.83062752e-02 -3.11566070e-02 -1.17329173e-01 -5.11690155e-02
  -8.85287598e-02 -2.18963567e-02  1.42986383e-02  4.44167741e-02
  -1.34815723e-02  7.43392333e-02  2.66382582e-02 -1.98762715e-02
   1.79191

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

#print("Sentence embeddings:")
#print(sentence_embeddings)

In [ ]:
sentences = ['Dr Edgar Chagwa Lungu will try to run for presidency in 2026', 'iVerify Zambia has firmly stated after hours and hours of discussion that Dr Edgar Chagwa Lungu is ready for General Elections in 2026']

encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

#print("Sentence embeddings:")
#print(sentence_embeddings)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cos = cosine_similarity(sentence_embeddings[0].reshape(1, -1), sentence_embeddings[1].reshape(1, -1))
cos

array([[0.67956126]], dtype=float32)

In [ ]:
import nltk
nltk.download('punkt')

text = "Dr. Edgar Chagwa Lungu will run for presidency in 2026. He is a doctor from the UPV \n and worked with Mr. Jones and J.K. Rowling in Hawaii."

a_list = nltk.tokenize.sent_tokenize(text)


print(a_list)

['Dr. Edgar Chagwa Lungu will run for presidency in 2026.', 'He is a doctor from the UPV \n and worked with Mr. Jones and J.K. Rowling in Hawaii.']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
frases = ['iVerify Zambia has fact-checked the claim by Zed Diary Facebook Page to establish if Mr. Edgar Chagwa Lungu had indeed stated in an interview that President Hakainde Hichilema had lied his way to power.',
 'Our verification process has found that indeed Mr. Lungu had said that some people lied their way to power, but did not refer to President Hakainde Hichilema by  name in his statement during the interview.',
 'Therefore, the claim by Zed Dairy  that Mr. Lungu had  stated that  President Hakainde Hichilema had lied his way into power, is misleading.',
 'It is false that former President of the Republic of Zambia Mr Edgar Chagwa Lungu accused the current Republican President, Mr. Hakainde Hichilema of being a hypocrite who does say one thing and does another.',
 'Mr. Lungu did not mention President Hichilema or any other alleged “hypocrite”.',
 'After the burial of the late former President Mr Rupiah Banda, while responding to a journalist’s question on lessons learnt from Mr Banda’s life, Mr Lungu said “we should learn to dialogue honestly with one another …there is no need for hypocrisy where people say good things but do bad things”.',
 'In a short video clip watched by iVerify, Mr. Lungu uses the word hypocrisy but does not mention President Hakainde Hichilema.',
 'It is true that Mr. Hakainde Hichilema won the Presidential election with a landslide victory.',
 'Although the Electoral Commission of Zambia formally declared him a winner in the early hours of 16th August 2021, some candidates had already confirmed his victory on 13th August 2021, a day after the General elections.',
 'They claimed their confirmation was based on results from polling stations, and went on to concede defeat, before the official results were announced, and endorsed the United Party for National Development (UPND) candidate, Hakainde Hichilema as a winner.',
 'However, regardless of the predictions made, the Electoral commission of Zambia is the only body that is legally mandated to declare a candidate as a president after following proper and orderly way of the electoral process.',
 'In the early hours of Monday 16th August, 2021, the Electoral Commission of Zambia declared Mr. Hakainde Hichilema as President elect, after consolidating results that kept him in the lead with the a total number 2,810,757 votes, followed by Mr. Edgar Chagwa Lungu with 1,814,201 votes.',
 'Furthermore on 16th August, 2021, outgoing President Mr. Edgar Chagwa Lungu conceded defeat at around 12hrs and congratulated the incoming president elect Mr. Hakainde Hichilema.',
 'The iVerify Zambia has determined as false the claim by the Koswe Facebook page that Zambia’s former Republican President, Dr Edgar Chagwa Lungu had  declared himself ready for the 2026 General Elections and that he was determined to remove the current Republican President, Mr. Hakainde Hichilema from office.',
 'The video clip alleged to be recent was taken 3 years ago during a memorial service for the late Republican President, Mr. Michael Chilufya Sata.',
 'iVerify Zambia contacted former Zambia’s Ambassador to Ethiopia and an aspiring candidate for the Presidency of the Patriotic Front party, Honorable Emmanuel Mwamba refuted the claims by Koswe, confirming that the video clip was indeed taken in 2021 during the memorial service for the late President Michael Chilufya Sata.',
 'The fact checking process has determined as misleading the claim that over 573 health workers have been fired.',
 'The previous Patriotic Front Government led by former President Edgar Chagwa Lungu, through support from the World Bank and other cooperating partners, contracted additional health workers to support the mainstream health staff in the fight against Covid-19 across the country for a period of three (3) months.',
 'This was to strengthen the Government’s response to the pandemic in the wake of an increase in cases and deaths during the third wave.',
 'The commencement of the contract was on the 22nd June, 2021, and it was set to end on the 24th September, 2021.',
 'The termination of the contract was foreseen, and can therefore not be considered ‘firing’.',
 'The iVerify fact checking team contacted some health workers who affirmed that their contracts were only for a period of three (3) months, and were to end on 24th September, 2021.',
 'As part of the verification, the iVerify fact checking process also reviewed some contract documents for the health workers, which affirmed the statement.',
 'The Fact-Checking process has determined as true that criminals have been involved in the illegal acts of destruction and stealing of public and private property across the country.',
 'The attacks were carried out on those believed to be affiliated to the former ruling party, the Patriotic Front.',
 'A video report by the Zambia National Broadcasting Corporation (ZNBC) analyzed by the Fact-Checking team confirm that several shops at the Edgar Chagwa Lungu Mall in Kitwe were broken into in the early hours of 16th August, 2021 and goods worth millions of Kwacha looted, as people were celebrating President Elect Hakainde Hichilema’s victory in the 12th August general election.',
 'In a separate incident, criminal acts were recorded on the 16th August, were unknown people vandalized mobile money transaction booths in Lusaka’s central business district immediately after the Eletoral Commission of Zambia (ECZ) officially announced UPND’s Hakainde Hichilema as the winner of Zambia’s General elections.',
 'The iVerify Fact-Checking team further fact checked a video report by Victoria Kayeye and Memory Nyambe from Diamond TV entitled ‘Post Elections,’ which confirmed an incident where the Patriotic Front Secretariat property was stolen and destroyed.',
 'Following the events, the Inspector General of Police Kakoma Kanganja  issued a statement where he directed all police officers to ensure that they act swiftly and bring to a stop the prevailing violence and criminal acts.',
 'According to a Wednesday, August 18, 2021, Daily Nation Newspaper, Vol.7 issue 3026, Police Spokesperson, Ms. Katongo noted that criminals were taking advantage of the victory celebrations and want to act as cadres.',
 'She further confirmed that 15 people have been arrested in connection with the violence and looting in Lusaka and ECL mall in Kitwe, and that the police are following up on culprits that took part in the looting.',
 'The fact checking process has established as inconclusive the claim by Mr. Mwanza that PF North Western Provincial Chairperson Mr. Jackson Kungo had been murdered by UPND cadres.',
 'While the death of Mr. Kungo was confirmed by the Police and other stakeholders, the fact checking process could not find any verifiable evidence that Mr Kungo was killed by UPND cadres and that he was delivering voter registers.',
 'In a statement on 12th August 2021, President Edgar Chagwa Lungu confirmed that PF Chairman Jackson Kungo died in the afternoon of 12th August 2021.',
 'The PF said at the time of his attack, Mr Kungo had been sent to distribute voter registers to the PF monitors at Kyawama polling station in Solwezi.',
 'However, contrary to that statement, the Northwestern Province Police Commissioner stated that Mr. Kungo went to vote at Kyawama polling station when it was suspected that he was carrying pre-marked ballot papers, before the residents pounced on him.',
 'A call with a polling monitor further disputes this allegation by the police that Mr Kungo was set to vote at Kyawama polling station as he was not registered to vote from there.',
 'There are therefor three different versions of Mr. Kungo’s presence at the polling station.',
 'iVerify cannot confirm that the group of voters are UPND cadres or affiliated to UPND or any other political party because there is no proof that the alleged mob was dressed in UPND regalia nor identified themselves as such.',
 'An analysis carried out by iVerify shows that one of the polling day offences by the Electoral Commission of Zambia (ECZ) includes putting up a political notice or sign or being clad in political regalia within a radius of 100 meters of a polling station during polling day.',
 'It would therefore render it difficult or close to impossible to have an individual in political regalia at a polling station.',
 'To confirm this, iVerify made a call to a polling assistant at Kyawama polling station who confirmed that no voter was allowed entry into the polling station whilst clad in or being in possession of political regalia or sign, conclusively disqualifying any claims of political identification.']

In [ ]:
sentences = ['Dr Edgar Chagwa Lungu will try to run for presidency in 2026']+frases

encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

#print("Sentence embeddings:")
#print(sentence_embeddings)

In [ ]:
lista = []
from sklearn.metrics.pairwise import cosine_similarity
for sentence in range(1, len(sentence_embeddings)):
  cos = cosine_similarity(sentence_embeddings[0].reshape(1, -1), sentence_embeddings[sentence].reshape(1, -1))
  if cos > 0.5:
    lista.append(sentences[sentence])

In [ ]:
lista

['Furthermore on 16th August, 2021, outgoing President Mr. Edgar Chagwa Lungu conceded defeat at around 12hrs and congratulated the incoming president elect Mr. Hakainde Hichilema.',
 'The iVerify Zambia has determined as false the claim by the Koswe Facebook page that Zambia’s former Republican President, Dr Edgar Chagwa Lungu had  declared himself ready for the 2026 General Elections and that he was determined to remove the current Republican President, Mr. Hakainde Hichilema from office.',
 'The previous Patriotic Front Government led by former President Edgar Chagwa Lungu, through support from the World Bank and other cooperating partners, contracted additional health workers to support the mainstream health staff in the fight against Covid-19 across the country for a period of three (3) months.']

In [ ]:
for sentence in sentences:
  if "Edgar" in sentence:
    print(sentence)
    print()

Dr Edgar Chagwa Lungu will try to run for presidency in 2026

iVerify Zambia has fact-checked the claim by Zed Diary Facebook Page to establish if Mr. Edgar Chagwa Lungu had indeed stated in an interview that President Hakainde Hichilema had lied his way to power.

It is false that former President of the Republic of Zambia Mr Edgar Chagwa Lungu accused the current Republican President, Mr. Hakainde Hichilema of being a hypocrite who does say one thing and does another.

In the early hours of Monday 16th August, 2021, the Electoral Commission of Zambia declared Mr. Hakainde Hichilema as President elect, after consolidating results that kept him in the lead with the a total number 2,810,757 votes, followed by Mr. Edgar Chagwa Lungu with 1,814,201 votes.

Furthermore on 16th August, 2021, outgoing President Mr. Edgar Chagwa Lungu conceded defeat at around 12hrs and congratulated the incoming president elect Mr. Hakainde Hichilema.

The iVerify Zambia has determined as false the claim by 

## Creating the class

In [ ]:
class Fact_checking_module:
  import nltk
  nltk.download('punkt')

  from transformers import AutoTokenizer, AutoModel
  import torch
  import torch.nn.functional as F

  # Load model from HuggingFace Hub
  tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
  model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

  def __init__(self):
    pass

  #Mean Pooling - Take attention mask into account for correct averaging
  def mean_pooling(cls, model_output, attention_mask):
      token_embeddings = model_output[0] #First element of model_output contains all token embeddings
      input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
      return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

  def add_evidences(self, claim:str, texts:list, threshold:float)->list:
    sentences = [claim]
    for text in texts:
      text = text.replace('\n', ' ')
      sent_list = nltk.tokenize.sent_tokenize(text)
      sentences = sentences + sent_list

    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    
    lista = []
    from sklearn.metrics.pairwise import cosine_similarity
    for sentence in range(1, len(sentence_embeddings)):
      cos = cosine_similarity(sentence_embeddings[0].reshape(1, -1), sentence_embeddings[sentence].reshape(1, -1))
      if cos > threshold:
        lista.append((sentences[sentence], float(cos)))

    return lista, sentences



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
fct = Fact_checking_module()
claim = 'Mr. Edgar Chagwa Lungu congratulated the incoming president elect Mr. Hakainde Hichilema.'
lista, sentences = fct.add_evidences(claim, [' '.join(frases)], 0.5)

In [ ]:
lista

[('iVerify Zambia has fact-checked the claim by Zed Diary Facebook Page to establish if Mr. Edgar Chagwa Lungu had indeed stated in an interview that President Hakainde Hichilema had lied his way to power.',
  0.5269086360931396),
 ('Our verification process has found that indeed Mr. Lungu had said that some people lied their way to power, but did not refer to President Hakainde Hichilema by  name in his statement during the interview.',
  0.5064297914505005),
 ('It is false that former President of the Republic of Zambia Mr Edgar Chagwa Lungu accused the current Republican President, Mr. Hakainde Hichilema of being a hypocrite who does say one thing and does another.',
  0.5009831190109253),
 ('It is true that Mr. Hakainde Hichilema won the Presidential election with a landslide victory.',
  0.6157991886138916),
 ('In the early hours of Monday 16th August, 2021, the Electoral Commission of Zambia declared Mr. Hakainde Hichilema as President elect, after consolidating results that kept